In [0]:
from delta.tables import *

In [0]:
df = spark.sql("select * from ete_project.silver.orders")
df.display()

In [0]:
df_dim_cust = spark.sql('select dimcustomerkey,customer_id as dim_customer_id from ete_project.gold.dimcustomers')

df_dim_prod = spark.sql('select product_id as dimproductkey,product_id as dim_product_id from ete_project.gold.dimproducts')


In [0]:
df_fact = df.join(df_dim_cust,df.customer_id == df_dim_cust.dim_customer_id,'inner').join(df_dim_prod,df.product_id == df_dim_prod.dim_product_id,'left')

df_fact_new = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')

In [0]:
df_fact_new.display()

In [0]:
if spark.catalog.tableExists("ete_project.gold.factorders"):
    dlt_obj = DeltaTable.forName(spark,'ete_project.gold.factorders')
    dlt_obj.alias('t').merge(df_fact_new.alias('s'), 't.order_id = s.order_id').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_fact_new.write.format('delta').option('path','abfss://sales@adls7428.dfs.core.windows.net/gold/factorders').saveAsTable('ete_project.gold.factorders')



In [0]:
%sql

select * from ete_project.gold.factorders;